In [60]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from time import time

np.random.seed(1337)

dftrain = pd.read_csv('/Users/laksh/Documents/fantasy/FantasySports/data/extras/train.csv', header=None)
dftest = pd.read_csv('/Users/laksh/Documents/fantasy/FantasySports/data/extras/test.csv', header=None)

In [61]:
df_train = dftrain.iloc[:, 1:]

scaler = StandardScaler()
# features = ['AvgPointsPerGame', 'atHome', '7GameAvg', 'FloorAvg']

X_train = scaler.fit_transform(df_train.iloc[:,:-1].values)
y_train = df_train.iloc[:,-1].values
#y_train_onehot = pd.get_dummies(df_train['4GameAvg']).values

In [62]:
df_test = dftest.iloc[:, 1:]

X_test = scaler.transform(df_test.iloc[:, :-1].values)
y_test = df_test.iloc[:, -1].values

In [64]:
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor(random_state=0, verbose=3)
model = model.fit(X_train, y_train)

y_prediction = model.predict(X_test)
print ("\naccuracy", np.sum(abs(y_prediction-y_test)<10) / float(len(y_test)))

building tree 1 of 10
building tree 2 of 10
building tree 3 of 10
building tree 4 of 10
building tree 5 of 10
building tree 6 of 10
building tree 7 of 10
building tree 8 of 10
building tree 9 of 10
building tree 10 of 10

accuracy 0.701123595506


[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.0s finished


In [69]:
pd.concat([pd.DataFrame(y_prediction), pd.DataFrame(y_test)],axis=1)

,0,0
0,35.040000,32.5
1,13.720000,13.0
2,33.670000,34.8
3,21.620000,25.8
4,0.924257,0.0
5,4.800000,0.0
6,2.962775,0.0
7,28.880000,29.5
8,19.850000,18.5
9,19.990000,13.2


In [27]:
from keras.models import Sequential
from keras.layers import Dense, Activation

start = time()

model = Sequential()
model.add(Dense(input_dim=4, output_dim=2))
model.add(Activation("softmax"))

model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])

#model.fit(X_train, y_train)

print ('\ntime taken %s seconds' % str(time() - start))


time taken 0.04978013038635254 seconds
